In [1]:
%matplotlib inline

(R-GCN): This type of network is one effort to generalize GCN 
to handle different relationships between entities in a knowledge base. 
To learn more about the research behind R-GCN, see `Modeling Relational Data with Graph Convolutional
Networks <https://arxiv.org/pdf/1703.06103.pdf>`_ 


The straightforward graph convolutional network (GCN) and 
`DGL tutorial <http://doc.dgl.ai/tutorials/index.html>`_) exploits
structural information of a dataset (that is, the graph connectivity) in order to improve the extraction of node representations. Graph edges are left as untyped.


A knowledge graph is made up of a collection of triples in the form
subject, relation, object. Edges thus encode important information and have their own embeddings to be learned. Furthermore, there may exist multiple edges among any given pair.

## Part 1: Entity Classification Preparation

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from functools import partial

class RGCNLayer(nn.Module):
    def __init__(self, in_feat, out_feat, num_rels, num_bases=-1, bias=None,
                 activation=None, is_input_layer=False):
        super(RGCNLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.is_input_layer = is_input_layer

        # sanity check
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels

        # weight bases in equation (3)
        self.weight = nn.Parameter(torch.Tensor(self.num_bases, self.in_feat,
                                                self.out_feat))
        if self.num_bases < self.num_rels:
            # linear combination coefficients in equation (3)
            self.w_comp = nn.Parameter(torch.Tensor(self.num_rels, self.num_bases))

        # add bias
        if self.bias:
            self.bias = nn.Parameter(torch.Tensor(out_feat))

        # init trainable parameters
        nn.init.xavier_uniform_(self.weight,
                                gain=nn.init.calculate_gain('relu'))
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(self.w_comp,
                                    gain=nn.init.calculate_gain('relu'))
        if self.bias:
            nn.init.xavier_uniform_(self.bias,
                                    gain=nn.init.calculate_gain('relu'))

    def forward(self, g):
        if self.num_bases < self.num_rels:
            # generate all weights from bases (equation (3))
            weight = self.weight.view(self.in_feat, self.num_bases, self.out_feat)
            weight = torch.matmul(self.w_comp, weight).view(self.num_rels,
                                                        self.in_feat, self.out_feat)
        else:
            weight = self.weight

        if self.is_input_layer:
            def message_func(edges):
                # for input layer, matrix multiply can be converted to be
                # an embedding lookup using source node id
                embed = weight.view(-1, self.out_feat)
                index = edges.data['rel_type'] * self.in_feat + edges.src['id']
                return {'msg': embed[index] * edges.data['norm']}
        else:
            def message_func(edges):
                w = weight[edges.data['rel_type']]
                msg = torch.bmm(edges.src['h'].unsqueeze(1), w).squeeze()
                msg = msg * edges.data['norm']
                return {'msg': msg}

        def apply_func(nodes):
            h = nodes.data['h']
            if self.bias:
                h = h + self.bias
            if self.activation:
                h = self.activation(h)
            return {'h': h}

        g.update_all(message_func, fn.sum(msg='msg', out='h'), apply_func)

Using backend: pytorch


Full R-GCN model defined

In [3]:
class Model(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels,
                 num_bases=-1, num_hidden_layers=1):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers

        # create rgcn layers
        self.build_model()

        # create initial features
        self.features = self.create_features()

    def build_model(self):
        self.layers = nn.ModuleList()
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for _ in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer()
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    # initialize feature for each node
    def create_features(self):
        features = torch.arange(self.num_nodes)
        return features

    def build_input_layer(self):
        return RGCNLayer(self.num_nodes, self.h_dim, self.num_rels, self.num_bases,
                         activation=F.relu, is_input_layer=True)

    def build_hidden_layer(self):
        return RGCNLayer(self.h_dim, self.h_dim, self.num_rels, self.num_bases,
                         activation=F.relu)

    def build_output_layer(self):
        return RGCNLayer(self.h_dim, self.out_dim, self.num_rels, self.num_bases,
                         activation=partial(F.softmax, dim=1))

    def forward(self, g):
        if self.features is not None:
            g.ndata['id'] = self.features
        for layer in self.layers:
            layer(g)
        return g.ndata.pop('h')

Import our dataset

In [4]:
import pandas as pd
import networkx as nx

bios = pd.read_csv('network.csv')
bios = bios.drop_duplicates()
bios["agent_A"] = bios["agA_ns"] + ':' + bios["agA_id"]
bios["agent_B"] = bios["agB_ns"] + ':' + bios["agB_id"]
#bios["edge_attr"] = tf.constant(bios['stmt_type'])

bios_graph_data = bios[['agent_A', 'agent_B', 'stmt_type']]

bios_graph_data.head()

,agent_A,agent_B,stmt_type
0,HGNC:12873,HGNC:11715,Complex
1,HGNC:11715,HGNC:12873,Complex
2,HGNC:12731,HGNC:19297,Complex
3,HGNC:19297,HGNC:12731,Complex
4,HGNC:6843,HGNC:2631,Activation


Encode our 'edge' relationships, and create a lookup dictionary. These correspond to different biology agent relationships like  methylation, inhibition, etc. 

In [5]:
import tensorflow as tf


#bios_graph_data['test'] = bios_graph_data.apply(lambda x: tf.convert_to_tensor(x.stmt_type), axis=1)
edge_relation_lookup = dict( zip( bios_graph_data['stmt_type'].astype('category').cat.codes, bios_graph_data['stmt_type'] ) )
edge_relation_lookup

{2: 'Complex',
 1: 'Activation',
 16: 'IncreaseAmount',
 17: 'Inhibition',
 13: 'Glycosylation',
 21: 'Phosphorylation',
 23: 'Ubiquitination',
 5: 'DecreaseAmount',
 7: 'Dephosphorylation',
 12: 'Gef',
 3: 'Conversion',
 18: 'Methylation',
 10: 'Farnesylation',
 15: 'Hydroxylation',
 22: 'Sumoylation',
 0: 'Acetylation',
 9: 'Deubiquitination',
 4: 'Deacetylation',
 14: 'GtpActivation',
 11: 'Gap',
 6: 'Demethylation',
 19: 'Myristoylation',
 8: 'Desumoylation',
 20: 'Palmitoylation'}

In [6]:
bios_graph_data['relation_category'] = bios_graph_data.stmt_type.astype('category').cat.codes
bios_graph_data.head()

<ipython-input-6-a22459170bbe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bios_graph_data['relation_category'] = bios_graph_data.stmt_type.astype('category').cat.codes


,agent_A,agent_B,stmt_type,relation_category
0,HGNC:12873,HGNC:11715,Complex,2
1,HGNC:11715,HGNC:12873,Complex,2
2,HGNC:12731,HGNC:19297,Complex,2
3,HGNC:19297,HGNC:12731,Complex,2
4,HGNC:6843,HGNC:2631,Activation,1


In the RGCN paper, they specify normalizing the edges. Here we will implement this manually since it was never documented in code how they did this. Other papers suggest (in the case where there is a hypothetical edge between node A and node B) taking the square root of node A centrality times node B centrality. 

First create node centrality dictionary

In [7]:
g_w_data = nx.from_pandas_edgelist(bios_graph_data,source='agent_A',target='agent_B', edge_attr=["relation_category"], create_using=nx.DiGraph())
dict1 = nx.degree_centrality(g_w_data)

def print_centrality(dict):
    for i,w in enumerate(sorted(dict, key=dict.get, reverse=True)):
        if (i < 15):
            print (w, dict[w])
        else:
            break
            
print_centrality(dict1)

HGNC:12932 0.06021357580883908
HGNC:620 0.05979065341509833
HGNC:8031 0.053552548107422285
HGNC:3236 0.04699725100444068
HGNC:3312 0.045569887925565655
HGNC:11998 0.04414252484669063
HGNC:12825 0.033516599703954326
HGNC:2553 0.03129625713681539
HGNC:6871 0.0312433918375978
HGNC:1100 0.031190526538380205
HGNC:3467 0.029816028758722773
HGNC:7200 0.02886445337280609
HGNC:8071 0.028811588073588495
HGNC:1771 0.027754282089236623
HGNC:11283 0.027754282089236623


In [8]:
import numpy as np

bios_graph_data['edge_norm'] = bios_graph_data.apply(lambda x: np.sqrt(dict1[x.agent_A]*dict1[x.agent_B]), axis=1)
bios_graph_data.head()

<ipython-input-8-0033c08ccb2e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bios_graph_data['edge_norm'] = bios_graph_data.apply(lambda x: np.sqrt(dict1[x.agent_A]*dict1[x.agent_B]), axis=1)


,agent_A,agent_B,stmt_type,relation_category,edge_norm
0,HGNC:12873,HGNC:11715,Complex,2,0.001123
1,HGNC:11715,HGNC:12873,Complex,2,0.001123
2,HGNC:12731,HGNC:19297,Complex,2,0.003247
3,HGNC:19297,HGNC:12731,Complex,2,0.003247
4,HGNC:6843,HGNC:2631,Activation,1,0.001942


We have now successfully created the edge normalization values. 

Now let's create the DGL graph object. I ended up with the following:


In [9]:
edge_src = bios_graph_data['agent_A'].astype('category').cat.codes.tolist()
edge_dst = bios_graph_data['agent_B'].astype('category').cat.codes.tolist()
g = DGLGraph((edge_src, edge_dst))

/Users/christinawang/.conda/envs/section13/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [10]:
print('#Nodes', g.number_of_nodes())
print('#Edges', g.number_of_edges())

#Nodes 18210
#Edges 199526


Double check a few methods to make sure the structure is correct

Get the in-degree of node 92:

In [11]:
g.in_degree(92)

/Users/christinawang/.conda/envs/section13/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.in_degree is deprecated. Please use DGLGraph.in_degrees
  return warnings.warn(message, category=category, stacklevel=1)


6

Get the successors of node 92:


In [12]:
g.successors(92)

tensor([12062, 12063])

Let's add edge attributes for the 'relational' part of R-GCN

In [13]:
num_rels = len(np.unique(bios_graph_data['relation_category'].tolist()))
num_classes=0
print('Number of unique biological interations/edge relations: ', num_rels)

Number of unique biological interations/edge relations:  24


In [14]:
bios_graph_data['relation'] = bios_graph_data.apply(lambda x: ('agent', x.stmt_type, 'agent'), axis=1)
bios_graph_data.head()

<ipython-input-14-df3a597349ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bios_graph_data['relation'] = bios_graph_data.apply(lambda x: ('agent', x.stmt_type, 'agent'), axis=1)


,agent_A,agent_B,stmt_type,relation_category,edge_norm,relation
0,HGNC:12873,HGNC:11715,Complex,2,0.001123,"(agent, Complex, agent)"
1,HGNC:11715,HGNC:12873,Complex,2,0.001123,"(agent, Complex, agent)"
2,HGNC:12731,HGNC:19297,Complex,2,0.003247,"(agent, Complex, agent)"
3,HGNC:19297,HGNC:12731,Complex,2,0.003247,"(agent, Complex, agent)"
4,HGNC:6843,HGNC:2631,Activation,1,0.001942,"(agent, Activation, agent)"


In [15]:
#edge_type = np.array(bios_graph_data['relation_category'])
edge_type = torch.tensor(bios_graph_data['relation_category'].to_numpy())
#edge_name_tuple = torch.tensor(bios_graph_data['stmt_type'])

g.edata['rel_type'] = edge_type
g.edata['etype'] = edge_type

In [16]:
#g.apply_edges(lambda edges: {'etype': edges})

In [17]:
len(edge_type)

199526

In [18]:
edge_norm = torch.tensor(bios_graph_data['edge_norm'].to_numpy())
g.edata['norm'] = edge_norm

In [19]:
len(edge_norm)

199526

In [20]:
g.edges()

(tensor([5396, 4777, 5309,  ..., 3404,  910, 3073]),
 tensor([ 1378,  2014,  5322,  ..., 13638, 11352, 11848]))

In [21]:
g.number_of_edges()

199526

In [22]:
g.edata

{'rel_type': tensor([ 2,  2,  2,  ..., 17, 16, 17], dtype=torch.int8), 'etype': tensor([ 2,  2,  2,  ..., 17, 16, 17], dtype=torch.int8), 'norm': tensor([0.0011, 0.0011, 0.0032,  ..., 0.0031, 0.0013, 0.0004],
       dtype=torch.float64)}

In [23]:
len(g)

/Users/christinawang/.conda/envs/section13/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)


18210

In [ ]:
# configurations
n_hidden = 199526 # number of hidden units
n_bases = -1 # use number of relations as number of bases
n_hidden_layers = 0 # use 1 input layer, 1 output layer, no hidden layer
n_epochs = 25 # epochs to train
lr = 0.01 # learning rate
l2norm = 0 # L2 norm coefficient


#unsure -took my best guess for these values
num_classes= 1
num_rels = len(np.unique(bios_graph_data['relation_category'].tolist()))


model = Model(len(g),    
              n_hidden, 
              num_classes,
              num_rels,
              num_bases=n_bases,
              num_hidden_layers=n_hidden_layers)

Create index for train, validation, and test

In [ ]:
# train_idx = bios_graph_data.index[0:5000]
# valid_idx = bios_graph_data.index[6000:7000]
# test_idx = bios_graph_data.index[5000:6000]

#not randomizing the index sampling for now for my sanity's sake
train_idx = bios_graph_data.index[0:10]
valid_idx = bios_graph_data.index[10:15]
test_idx = bios_graph_data.index[15:20]

Train and run classification

In [ ]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)

print("start training...")
model.train()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    logits = model.forward(g)
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])
    loss.backward()

    optimizer.step()

    train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx])
    train_acc = train_acc.item() / len(train_idx)
    val_loss = F.cross_entropy(logits[val_idx], labels[val_idx])
    val_acc = torch.sum(logits[val_idx].argmax(dim=1) == labels[val_idx])
    val_acc = val_acc.item() / len(val_idx)
    print("Epoch {:05d} | ".format(epoch) +
          "Train Accuracy: {:.4f} | Train Loss: {:.4f} | ".format(
              train_acc, loss.item()) +
          "Validation Accuracy: {:.4f} | Validation loss: {:.4f}".format(
              val_acc, val_loss.item()))